In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split,cross_val_score, GridSearchCV
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score

In [ ]:
### stevejobs.txt

In [ ]:
# 불러오기
with open('../data/stevejobs.txt', 'r', encoding='utf8') as f:
    rows=f.readlines()
    lines=[row for row in rows]
text=' '.join(lines)

In [ ]:
# 1-1.텍스트 전처리(정규화): 클렌징
import re
compile=re.compile('[^ a-zA-Z0-9\.]+')  #[a-zA-Z]+
text=compile.sub('',text).lower()

In [ ]:
# 1-2.토큰화(문장)
import nltk
nltk.download('punkt')
sentences=nltk.sent_tokenize(text=text)

In [ ]:
# 토큰화(단어) extend: 리스트로 
word_token=[]
for sentence in sentences:
    words=nltk.word_tokenize(sentence)
    word_token.extend(words)
print(word_token[:3])

In [ ]:
# 토큰화(문단 --> 단어) append: 2차원 데이터프레임으로
word_token2=[]
for sentence in sentences:
    words=nltk.word_tokenize(sentence)
    word_token2.append(words)
print(word_token2[:3])

In [ ]:
# 토큰화 함수: 문서의 모든 단어
from nltk import sent_tokenize, word_tokenize

def tokenize_text(text):
    sentences = nltk.sent_tokenize(text)
    word_tokens = [nltk.word_tokenize(sentense) for sentense in sentences]
    return word_tokens
word_tokens = tokenize_text(text)
print(word_tokens[:2])

In [ ]:
# 1-3.불용어(stop_words) 제거
nltk.download('stopwords')
stopwords=nltk.corpus.stopwords.words('english')

all_tokens=[]
for sentence in word_tokens:
    filtered_words=[]
    for word in sentence:
        if word not in stopwords:
            filtered_words.append(word)
    all_tokens.append(filtered_words)
all_tokens[:1]

In [ ]:
# 1-4. 어근(형태소) 추출: Stemming/ Lemmatization -영어에서만 사용
from nltk import LancasterStemmer
stemmer=LancasterStemmer()
stemmer.stem('working')

In [ ]:
# 2.피처 벡터화/축출(BOW : CountVectorizer/TfidfVectorizer)
# 2-1. 카운트 기반의 벡터화: CountVectorizer
# 셋트 만들기
from sklearn.datasets import fetch_20newsgroups
train_news=fetch_20newsgroups(subset='train', remove=('headers','footers','quotes'), random_state=156)
X_train=train_news.data
y_train=train_news.target
test_news=fetch_20newsgroups(subset='train', remove=('headers','footers','quotes'), random_state=156)
X_test=test_news.data
y_test=test_news.target

# 벡처화
from sklearn.feature_extraction.text import CountVectorizer
cnt_vct=CountVectorizer()
cnt_vct.fit(X_train)   # y값 없음!!
X_train_cnt_vct=cnt_vct.transform(X_train)
X_test_cnt_vct=cnt_vct.transform(X_test)

# 3.모델로 분류
from sklearn.linear_model import LogisticRegression
lr_clf=LogisticRegression(solver='liblinear')
lr_clf.fit(X_train_cnt_vct, y_train)
pred=lr_clf.predict(X_test_cnt_vct)
print('예측 정확도 : ', accuracy_score(y_test, pred))

In [ ]:
# 2-2. TF-IDF: TfidfVectorizer
# (자주 나오는 단어에 가중치/모든 문서에서 자주나오는 단어에 패털티)
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vct=TfidfVectorizer()
tfidf_vct.fit(X_train)
X_train_tfdf_vct=tfidf_vct.transform(X_train)
X_test_tfdf_vct=tfidf_vct.transform(X_test)

# 3.모델로 분류
from sklearn.linear_model import LogisticRegression
lr_clf=LogisticRegression(solver='liblinear')
lr_clf.fit(X_train_tfidf_vct, y_train)
pred=lr_clf.predict(X_test_tfidf_vct)
print('예측 정확도 : ', accuracy_score(y_test, pred))

In [ ]:
# 3-1. GridSearchCV로 파라미터 성능 향상
params={'C':[0.01, 0.1, 1, 5, 10]}
gr_lr_clf = GridSearchCV(lr_clf,param_grid=params,cv=3,scoring='accuracy', verbose=1)
gr_lr_clf.fit(X_train_tfdf_vct , y_train)
print('LogisticRegression의 최적 파라미터: ', gr_lr_clf.best_params_)

# 최적 C값으로 예측, 정확도 평가
pred=gr_lr_clf.predict(X_test_tfdf_vct)
print('예측 정확도 : ', accuracy_score(y_test, pred))

In [ ]:
# pipeline으로 만들기:TF-IDF 벡터화, GridSearchCV 최적찾기
from sklearn.datasets import fetch_20newsgroups
train_news=fetch_20newsgroups(subset='train', remove=('headers','footers','quotes'), random_state=156)
X_train=train_news.data
y_train=train_news.target
test_news=fetch_20newsgroups(subset='train', remove=('headers','footers','quotes'), random_state=156)
X_test=test_news.data
y_test=test_news.target

from sklearn.pipeline import Pipeline

pipeline=Pipeline([('tfdf_vect', TfidfVectorizer(stop_words='english')),
                   ('lr_clf', LogisticRegression())])
params={'tfdf_vect__ngram_range': [(1,1),(1,2)],
        'tfdf_vect__max_df': [100,200,300],
        'lr_clf__C': [1,5,7]}
grid_cv_pipe=GridSearchCV(pipeline, param_grid=params,cv=3,scoring='accuracy')
grid_cv_pipe.fit(X_train,y_train)
print('최적 파라미터: ', grid_cv_pipe.best_params_)
pred=gr_lr_clf.predict(X_test)
print('예측 정확도 : ', accuracy_score(y_test, pred))

In [ ]:
# 한글 단어 토큰
from konlpy.tag import Okt
okt = Okt()
words = []
for sentence in sentences:
    word = okt.morphs(sentence)
    words.append(word)
print(words[:2])
test_text = '나는 정말로 파이썬을 좋아한다. 아니 머신러닝을 더 좋아한다.'
print('normalize :', okt.normalize(test_text)) # 문장으로 추출
print('morphs :', okt.morphs(test_text))       # 구문 분석
print('nouns :', okt.nouns(test_text))         # 명사만
print('phrases :', okt.phrases(test_text))     # 구문
print('pos :', okt.pos(test_text))             # 품사와 함께 값고 함께 

#### 감성 분석: 지도학습, 비지도학습

In [ ]:
# 영화 평가 분석// 지도학습 기반  (1) CountVectorizer + pipeline
review_df = pd.read_csv('data/labeledTrainData.tsv', header=0, sep="\t", quoting=3)

In [ ]:
import re
review_df['review'] = review_df['review'].str.replace('<br />',' ')
review_df['review'] = review_df['review'].apply( lambda x : re.sub("[^a-zA-Z]", " ", x) )

In [ ]:
y_df=review_df['sentiment']
X_df=review_df.drop(['id','sentiment'], axis=1, inplace=False)
X_train, X_test, y_train, y_test= train_test_split(X_df, y_df, test_size=0.3, random_state=156)
print(X_train.shape, X_test.shape)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score

pipeline = Pipeline([('cnt_vect', CountVectorizer(stop_words='english',ngram_range=(1,2) )),
                     ('lr_clf', LogisticRegression(solver='liblinear', C=10))])

In [ ]:
pipeline.fit(X_train['review'], y_train)
pred = pipeline.predict(X_test['review'])
pred_probs = pipeline.predict_proba(X_test['review'])[:,1]
print(f'예측 정확도는 {accuracy_score(y_test ,pred):.4f}')
print(f'ROC-AUC는 {roc_auc_score(y_test, pred_probs):.4f}')

#### 영화 평가 분석// 지도학습 기반 (2) 한글

In [ ]:
# 참고
# df['content']=df['content'].apply(lambda x : re.sub(r"\d+","",x))
# import re
# def cleaning(text):
#     p = re.compile("[^ ㄱ-ㅣ가-힣\.]+")
#     result = p.sub('',text)
#     return result
# df['content']=df['content'].apply(cleaning)

In [ ]:
# 조사 지우는 함수
def ok_not_josa(text):
    filtered_words=[]
    for word, type_ in result:
        if type_ !='Josa':   # 또는 (type_ =='Noun' or 'Verb')
            filtered_words.append(word)
    return filtered_words      # 문자로 받아서( text), 리스트(filtered_words)롷 반환

In [ ]:
df=pd.read_csv('data/네이버 스마트스토어 리뷰.csv')

import re
df['content'] = df['content'].apply(lambda x : re.sub('[^ ㄱ-ㅣ가-힣]+', '', x))

from konlpy.tag import Okt
okt = Okt()
def okt_tokenizer(text):
    tokens_ko = okt.morphs(text, stem = True)
    return tokens_ko

with open('data/stopword.txt','r',encoding='utf-8') as f:
    word = f.read()
stopwords = word.split('\n')

In [ ]:
X_train, X_test, y_train, y_test=train_test_split(df['content'],df['score'], test_size=0.2, random_state=0 ) 

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vct=TfidfVectorizer(tokenizer=okt_tokenizer, stop_words=stopwords, max_features=5000)
tfidf_vct.fit(X_train)
X_train_tf=tfidf_vct.transform(X_train)
X_test_tf=tfidf_vct.transform(X_test)

In [ ]:
lr_clf=LogisticRegression(solver='liblinear')
lr_clf.fit(X_train_tf, y_train)
pred=lr_clf.predict(X_test_tf)
print('예측 정확도 : ', accuracy_score(y_test, pred))

#### LDA 토픽 모델링 (1): CountVectorizer만 허용

In [ ]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

In [ ]:
cats = ['rec.motorcycles', 'rec.sport.baseball', 'comp.graphics', 
    'comp.windows.x', 'talk.politics.mideast', 
    'soc.religion.christian', 'sci.electronics', 'sci.med' ]
news_df= fetch_20newsgroups(subset='all', remove=('headers', 'footers', 'quotes'), categories=cats, random_state=0)

In [ ]:
count_vect = CountVectorizer(max_df=0.95,max_features=1000, min_df=2,stop_words='english', token_pattern = '[a-zA-Z]+',ngram_range=(1,2))
feat_vect = count_vect.fit_transform(news_df.data)

lda = LatentDirichletAllocation(n_components=8, random_state=0)
lda.fit(feat_vect)
print(lda.components_.shape)
print(lda.components_)  # 볼 수 없음, 숫자만 가득

In [ ]:
def display_topics(model, feature_names, no_top_words):
    for topic_index, topic in enumerate(model.components_):
        print('Topic #',topic_index)
        topic_word_indexes = topic.argsort()[::-1]
        top_indexes=topic_word_indexes[:no_top_words]
        feature_concat = ' '.join([feature_names[i] for i in top_indexes])
        print(feature_concat)
feature_names = count_vect.get_feature_names_out()
display_topics(lda, feature_names, 15)     # 가장 연관도 높은 word 15개 보기

#### LDA 토픽 모델링-(2) 한글

In [ ]:
df = pd.read_csv('data/petition.csv')
cats = ['정치개혁', '인권/성평등', '안전/환경', '교통/건축/국토', '육아/교육']
df_cats = df[df['category'].isin(cats)]
df_samples = df_cats.sample(frac=0.05, random_state = 0)
df_samples['category'].value_counts()

In [ ]:
import re
df_samples['content'] = df_samples['content'].apply(lambda x : re.sub('[^ ㄱ-ㅣ가-힣]+', '', x))

from konlpy.tag import Okt
okt = Okt()
def okt_tokenizer(text):
    tokens_ko = okt.morphs(text, stem = True)
    return tokens_ko

with open('data/stopword.txt','r',encoding='utf-8') as f:
    word = f.read()
stopwords = word.split('\n')

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vect = TfidfVectorizer(max_df = 0.9,max_features = 1000,min_df = 2, ngram_range = (1, 2),tokenizer = okt_tokenizer,stop_words = stopwords)
tfidf_df = tfidf_vect.fit_transform(df_samples['content'])

from sklearn.decomposition import LatentDirichletAllocation
lda = LatentDirichletAllocation(n_components = 5, random_state = 0)
lda.fit(tfidf_df)
print(lda.components_.shape)
print(lda.components_)

In [ ]:
feature_names = tfidf_vect.get_feature_names_out()
display_topics(lda, feature_names, 10)

In [ ]:
## LDA 토픽 모델링-(3) 한글/ 하나의 주제에서 주요 주제 뽑기

In [ ]:
df = pd.read_csv('data/petition.csv')

In [70]:
cats = ['육아/교육']
df_cats = df[df['category'].isin(cats)]
df_samples = df_cats.sample(frac=0.5, random_state = 0)
df_samples['content'] = df_samples['content'].apply(lambda x : re.sub('[^ ㄱ-ㅣ가-힣]+', '', x))

In [ ]:
tfidf_vect = TfidfVectorizer(max_df = 0.9, max_features = 1000,min_df = 2, ngram_range = (1, 2),tokenizer = okt_tokenizer,stop_words = stopwords)
tfidf_df = tfidf_vect.fit_transform(df_samples['content'])   

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation
lda = LatentDirichletAllocation(n_components = 5, random_state = 0)    
lda.fit(tfidf_df)
feature_names = tfidf_vect.get_feature_names_out()
display_topics(lda, feature_names, 15)    # 토픽보기 함수 display_topics  15개

In [ ]:
lda.components_.argsort()[::-1][:5]

In [ ]:
lda_df=pd.DataFrame(lda.components_, columns=tfidf_vect.get_feature_names_out())
lda_df

#### KMeans 군집

In [ ]:
# 앞에 이미: 불러오기/ 클리닝/ TfidfVectorizer 벡터화,학습,변환

from sklearn.cluster import KMeans
km_cluster = KMeans(n_clusters = 4, max_iter = 10000, random_state = 0)
km_cluster.fit(tfidf_df)
cluster_label = km_cluster.labels_
clust_df = df_samples[['content', 'category']]
clust_df['cluster'] = cluster_label
clust_df.head(3)

In [ ]:
# 군집별 top n 핵심단어, 그 단어의 중심 위치 상대값, 대상 파일명을 반환
def get_cluster_details(cluster_model, cluster_data, feature_names, clusters_num, top_n_features=10):
    cluster_details = {}
    centroid_feature_ordered_ind = cluster_model.cluster_centers_.argsort()[:,::-1]
    for cluster_num in range(clusters_num):
        cluster_details[cluster_num] = {}
        cluster_details[cluster_num]['cluster'] = cluster_num
        top_feature_indexes = centroid_feature_ordered_ind[cluster_num, :top_n_features]
        top_features = [ feature_names[ind] for ind in top_feature_indexes ]
        top_feature_values = cluster_model.cluster_centers_[cluster_num, top_feature_indexes].tolist()
        cluster_details[cluster_num]['top_features'] = top_features
        cluster_details[cluster_num]['top_features_value'] = top_feature_values
    return cluster_details

In [ ]:
# 보기좋게 함수  get_cluster_details 를 표현하는 함수
def print_cluster_details(cluster_details):
    for cluster_num, cluster_detail in cluster_details.items():
        print('# Cluster {0}'.format(cluster_num))
        print('Top features:', cluster_detail['top_features'])
clust_centers = km_cluster.cluster_centers_

# 위 두 함수 불러오기
feature_names = tfidf_vect.get_feature_names_out()
cluster_details = get_cluster_details(cluster_model=km_cluster, cluster_data=clust_df, 
                                      feature_names=feature_names, clusters_num=4, top_n_features=10 )
print_cluster_details(cluster_details)